In [19]:
import tensorflow as tf
import numpy as np
import os
import time

In [20]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [21]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [22]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [23]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


In [24]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [25]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [26]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [27]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [28]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [29]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [30]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [31]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [32]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [33]:
seq_length = 100
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [34]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [35]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [36]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [37]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [38]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [39]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [40]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [41]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [42]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [43]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [44]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [45]:
sampled_indices

array([39, 15, 24, 37, 32, 64, 10, 54,  5, 29, 44, 55, 61, 43, 53, 58, 47,
        6,  3, 52, 41,  3, 27, 14, 65,  3, 62, 57,  0, 42, 13,  1, 47, 12,
       64, 17, 23, 50, 52, 22, 41, 16, 27, 44, 29, 30, 57,  6, 48, 43, 31,
       21, 48, 25,  8,  4, 23, 45, 45, 51, 23, 35,  7, 64, 17, 56, 28,  4,
       34, 44, 37, 58, 29, 65, 62, 36, 18, 62, 53, 41, 34,  8, 15, 65, 51,
       19, 30,  8, 36, 52, 22, 32, 34, 14, 40, 15, 30, 11, 13,  3])

In [46]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ings you'll say a beggar nay.\n\nGLOUCESTER:\nIt is too heavy for your grace to wear.\n\nYORK:\nI weigh it"

Next Char Predictions:
 b"ZBKXSy3o&Pepvdnsh'!mb!NAz!wr[UNK]c?\nh;yDJkmIbCNePQr'idRHiL-$JfflJV,yDqO$UeXsPzwWEwnbU-BzlFQ-WmISUAaBQ:?!"


In [47]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1904664, shape=(), dtype=float32)


In [48]:
tf.exp(example_batch_mean_loss).numpy()

66.05359

In [49]:
model.compile(optimizer='adam', loss=loss)

In [50]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [51]:
EPOCHS = 20

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 13s 53ms/step - loss: 2.7260
Epoch 2/20
172/172 [==============================] - 10s 51ms/step - loss: 1.9981
Epoch 3/20
172/172 [==============================] - 11s 52ms/step - loss: 1.7260
Epoch 4/20
172/172 [==============================] - 11s 53ms/step - loss: 1.5637
Epoch 5/20
172/172 [==============================] - 11s 54ms/step - loss: 1.4630
Epoch 6/20
172/172 [==============================] - 12s 55ms/step - loss: 1.3938
Epoch 7/20
172/172 [==============================] - 12s 56ms/step - loss: 1.3418
Epoch 8/20
172/172 [==============================] - 11s 56ms/step - loss: 1.2976
Epoch 9/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2566
Epoch 10/20
172/172 [==============================] - 11s 57ms/step - loss: 1.2195
Epoch 11/20
172/172 [==============================] - 11s 57ms/step - loss: 1.1810
Epoch 12/20
172/172 [==============================] - 12s 58ms/step - loss: 1.1418
E

In [52]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [53]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [54]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
The queen is comfort; which are the
dire to-morrow are you by confession; yet,
Even for the father of a wife:
Now my good friend proclamation, here rid death
With head and trumpets, are towards Counsel:
My treschange enjoy, thou stendge whips?
Here's Lucentio.

LUCIO:
This is the chair,
She is my mouths, he will shake your tears:
Who now is he was lest his son so weary
That rooted to find Juliet castle's warge
Till the wisdom of the eagle's head
And has The queen's soon-heady-hoodings. Upon
Lict reverence, which else departs make trush.

VOLUMNIA:
No, by an under-take much.
York, good morrow, Ratcliff!-
MARCIUS:
A boldness of the Earl of Mercutio?

TRANIO:
Master, master! how far inity may never growind
To be o'er eye's right from him; or she, perchavio,
And these eyes stempt provosting of;
But I am sure of; upon your honour.

Nurse:
Her oath, I hear her reciond from his blood,
And from my life closed by the quarrel of the fox
And shows a man of chief; a bawd of revenge.

CLIFFO

In [55]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThe gates made good, my lord; that it comes home\nThat valiant in the walls, and try sword, one more\nWisdom of the envious daughters: here thou art\nAn hour shef blood i' the world's; and thou hast heard\nsneep doth cavorrow makes me wed.\n\nBALONAS:\nAnd Master Calian: 'tis no last to hinder of him.\n\nESCALUS:\nMercutio!\n\nLUCIO:\nGood my friends; good monours, nothing;\nOne that Baptista, cried to Romeo,\nAlike and thus royal age.\n\nRICHARD:\nRomeo is ease to wife; who hath got in's ears.\n\nSecond Servingman:\n\nPage:\nThis shall shake off guilty of good sister\nTo execute the tastly through all the\nhabit, served with bestraight:\nGive me my bond Mortaged, did men blood cut\nof delay, yet marry her noble across;\nAlas, alas! shalt thou know my firm\nI should do three look for pactifugh.\n\nMERCUTIO:\n\nANGELO:\nYet here that is not known to be his soldiers;\nAnd I had on the violet in the fair of it.\n\nWESTMORELAND:\nChose, how let me never fetch out of y

In [56]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [57]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
The other for a manfor, stand upon!
Thou straight, my lord, to give them in;
and I will steal, alon


In [58]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [59]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [60]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [61]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 58ms/step - loss: 2.7208


In [62]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1839
Epoch 1 Batch 50 Loss 2.0628
Epoch 1 Batch 100 Loss 1.9504
Epoch 1 Batch 150 Loss 1.8178

Epoch 1 Loss: 1.9849
Time taken for 1 epoch 12.93 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8117
Epoch 2 Batch 50 Loss 1.7447
Epoch 2 Batch 100 Loss 1.6887
Epoch 2 Batch 150 Loss 1.6905

Epoch 2 Loss: 1.7064
Time taken for 1 epoch 11.50 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5821
Epoch 3 Batch 50 Loss 1.5966
Epoch 3 Batch 100 Loss 1.5162
Epoch 3 Batch 150 Loss 1.5285

Epoch 3 Loss: 1.5466
Time taken for 1 epoch 11.29 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4563
Epoch 4 Batch 50 Loss 1.4625
Epoch 4 Batch 100 Loss 1.4883
Epoch 4 Batch 150 Loss 1.4610

Epoch 4 Loss: 1.4483
Time taken for 1 epoch 10.94 sec
_____________________________________________________________________